# memebox comment crawling

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### get page id from url

In [2]:
def page_id(url):
    return url.split("page/")[1].split("?")[0]

In [3]:
url = "http://www.memebox.com/page/2576?source=1152"    
page_id(url)

'2576'

### get last page from page id

In [4]:
def last_page(page_id):
    url = "http://www.memebox.com/page/review?pageId=" + str(page_id) + "&contentViewPickOption=2"
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    pages = dom.select(".mypaging a")
    pages = len(pages)-4
    return pages

In [5]:
last_page(2576)

3

### get one page comments

In [6]:
def one_page_comments(page_id, page=1):
    
    url = "http://www.memebox.com/page/review?pageId=" + str(page_id) + "&contentViewPickOption=2&page=" + str(page)
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    comments = dom.select(".view")
    
    comments_list = []
    
    for comment in comments:
        star = comment.select_one("em").get("class")[0]
        content = comment.select_one(".textArea").text.strip()
        comments_list.append((star, content))
        
    return comments_list

In [7]:
comments = one_page_comments(2576)
len(comments)

10

### total comments from page url

In [8]:
def total_comments(url):
    p_id = page_id(url)
    pages = last_page(p_id)
    
    total_comments = []
    
    for page in range(1, pages+1):
        total_comments += one_page_comments(p_id, page)
            
    return total_comments

In [9]:
url = "http://www.memebox.com/page/2576?source=1152"    
result_comments = total_comments(url)

### make dataframe

In [10]:
star_list, comment_list = zip(*result_comments)
df = pd.DataFrame(columns=["star","comment"])
df["star"] = star_list
df["comment"] = comment_list
df

,star,comment
0,five,국내21호쓰는데요 2호를 샀어요. 셰딩용으로 써야하나 고민중입니다. 아니면 밝은 베...
1,five,색상이 환해요원래쓰던 컬러와 믹스해서 쓰는데 화사해보이고 아주 좋습니다.
2,five,배송때문에 늦게받았지만 맘에듭니다 적당히 촉촉하고 밀착이 좋아요아쉬운건 자외선 차단...
3,five,파운데이션만보면환장하는스타일이라화사하게하나더구입해야겠어요내통장너덜너덜
4,threeHalf,UHD가 좀 건조한것 같아서 구매했어요~11호라 밝고 화사하네요^^
5,five,1호샀어요 ㅎㅎ 이게 그렇게 좋다길래 구매했는데 기대되네요
6,five,언니가 쓰던제품인데 촉촉하고 좋다고 해서 한번 쓰려고여
7,five,원래 쓰던 제품이에요 ㅋㅋㅋ 가볍고 촉촉해요 대신 커버력은 좀 떨어져요
8,threeHalf,"배송은 오래걸렸지만, 제품은 만족스러웠어요!촉촉하다고해서 커버력은 기대하지않았는데,..."
9,five,벌써 두개째 구매해서 사용하고 있습니다.이번엔 미미박스를 통해 조금이라도 저렴하게 ...
